# Tutorial on Subgroup Discovery and SISSO

In this tutorial, we will utilize the subgroup-discovery (SGD) and the sure-independence screening and sparsifying operator (SISSO) symbolic-regression approaches to obtain interpretable models for materials' properties or functions. 

The tutorial is based on the approach and data of the following publication:

<div style="padding: 1ex; margin-top: 1ex; margin-bottom: 1ex; border-style: dotted; border-width: 1pt; border-color: blue; border-radius: 3px;">
J. M. Mauß, K. Kley, R. Khobragade, N-K. Tran, J. De Bellis, F. Schüth, M. Scheffler, L. Foppa: <span style="font-style: italic;">"Modeling Time-On-Stream Catalyst Reactivity in the Selective Hydrogenation of Concentrated Acetylene Streams Under Industrial Conditions via Experiments and AI"</span>, ChemRxiv,10.26434/chemrxiv-2025-vf7hd-v2 (2025) <a href="https://chemrxiv.org/engage/chemrxiv/article-details/67ea84986dde43c908d79032" target="_blank">[Paper]</a>.
</div>


# 1. Introduction

## Subgroup Discovery (SGD)
The SGD approach is based on a data set containing a target quantity of interest and many candidate descriptive parameters (primary features), whose values are known for all the samples in the data set. The target quantity can be the materials property of interest. The candidate descriptive parameters, in turn, characterize the materials and the underlying processes that are potentially relevant for the materials property. 

SGD starts by generating a pool of propositions, statements about the candidate descriptive parameters that apply only to a portion of the data set. For the case of continuous parameters, the propositions are inequalities constraining the values of the descriptive parameters. Then, SGD identifies selectors, statements formed by a number of propositions combined via the “AND” connector (denoted “∧”), that result in the selection of SGs of data points associated with the most outstanding distributions of the target values. The quality function $Q(SG,\widetilde{P})$ measures how outstanding a SG is. It has the form 

\begin{equation}
 Q(SG,\widetilde{P}) = \frac{s(SG)}{s(\widetilde{P})}* u(SG,\widetilde{P})
\end{equation} 

where the first term, the coverage, contains the ratio between the number of data points in the subgroup, $s(SG)$, and the total number of data points in the entire dataset, $s(\widetilde{P})$. This coverage term controls the subgroup size and prevents that very small SGs with little statistical significance are selected. The second term, $u(SG,\widetilde{P})$, is called utility function and it measures the dissimilarity between the SG and the entire data set, denoted $\widetilde{P}$. We refer to the dataset as $\widetilde{P}$ in order to stress that it is a sample of the entire, unknown population $P$. Many utility functions can be used in SGD depending on the scientific question and desired values of the target quantity. In this tutorial, we will compare the results obtained with different utility functions.


The outcome of the SGD analysis are the selectors resulting in the highest $Q(SG,\widetilde{P})$ values, i.e., in the most outstanding SGs. They can be seen as rules describing the SG behavior. The parameters entering these propositions are, in turn, the key, most relevant, parameters associated with the SG, out of all the initially offered candidate descriptive parameters. Because the SG search is performed by maximizing a function measuring how outstanding specific subselections of data points are, this approach identifies a local behavior.

The SGD approach is presented in more details in:

<div style="padding: 1ex; margin-top: 1ex; margin-bottom: 1ex; border-style: dotted; border-width: 1pt; border-color: blue; border-radius: 3px;"> B. R. Goldsmith, M. Boley, J. Vreeken, M. Scheffler, L. M. Ghiringhelli: <span style="font-style: italic;">Uncovering structure-property relationships of materials by subgroup discovery.</span>, New J. Physics 19, 013031 (2017) <a href="https://doi.org/10.1088/1367-2630/aa57c2" target="_blank">[PDF]</a> .
</div>

<div style="padding: 1ex; margin-top: 1ex; margin-bottom: 1ex; border-style: dotted; border-width: 1pt; border-color: blue; border-radius: 3px;"> M. Boley, B. R. Goldsmith,  L. M. Ghiringhelli, J. Vreeken: <span style="font-style: italic;">Identifying consistent statements about numerical data with dispersion-corrected subgroup discovery</span>, Data Min. Knowl. Discov. 31, 1391 (2017) <a href="https://doi.org/10.1007/s10618-017-0520-3" target="_blank">[PDF]</a> .
</div>

In this tutorial, we use the SGD algorithm as implemented in realkd, available in https://bitbucket.org/realKD/.


## Sure-Independence Screening and Sparsifying Operator (SISSO)

Symbolic regression (SR) identifies nonlinear analytical expressions relating a target property to key parameters, out of many offered candidate descriptive parameters (primary features). Thus, it is a possible avenue for linking physical reasoning and data-centric approaches when modelling materials properties and functions. The candidate descriptive parameters are typically physical quantities that are related to possible underlying processes governing the property. Because the key parameters can be directly identified by inspecting the analytical expressions, the models resulting from the SR analysis are interpretable. Another advantage of SR is that it can learn a representation for the property of interest based on data sets that are considered small (e.g., $10^2$ data points or less) compared to the trainig datasets of widely used artificial-intelligence methods such as artificial neural networks. Thus, SR is particularly suited for experimental materials-science problems, in which consistent and well characterized sets of materilas might be scarce. 

SISSO combines SR with compressed sensing, and it is a deterministic approach to obtain the mentioned analytical expressions. SISSO starts with the collection of physical input parameters, candidate descriptive parameters or primary features. Then, a more expansive pool of expressions is iteratively built by exhaustively applying a set of mathematical operators to both the primary features and previously generated expressions. This step is referred to as the feature-creation step. The number of recursive applications of the operators used to construct the pool of expressions is called the rung (denoted $q$). Finally, compressed sensing is used to identify the best $D$-dimensional linear model by performing an $l_0$ regularization on a subspace $S$ of all generated expressions, where $S$ is selected using SIS. The Pearson correlation is used as the projection score. 

The outcome of the SISSO analysis is a low $D$-dimensional descriptor vector containing, as components, the expressions selected from the pool of expressions. A SISSO-derived model for a property $S$ has the form

\begin{equation}
 S^{\mathrm{SISSO}} = \sum_{i=0}^{D} c_i d_i
\end{equation} 

where $c_i$ are fitting coefficients and $d_i$ are the descriptor components. We note that $q$ and $D$ are *hyperparameters* of the SISSO model. In this tutorial, we will utilize a cross-validation scheme in order to determine them.

The SISSO algorithm is presented in more details in:

<div style="padding: 1ex; margin-top: 1ex; margin-bottom: 1ex; border-style: dotted; border-width: 1pt; border-color: blue; border-radius: 3px;">
R. Ouyang, S. Curtarolo, E. Ahmetcik, M. Scheffler, L. M. Ghiringhelli: <span style="font-style: italic;">SISSO: a compressed-sensing method for identifying the best low-dimensional descriptor in an immensity of offered candidates</span>, Phys. Rev. Materials  2, 083802 (2018) <a href="https://journals.aps.org/prmaterials/abstract/10.1103/PhysRevMaterials.2.083802" target="_blank">[PDF]</a>.
</div>

## Focused AI Approach Combining SGD and SISSO

The two-step focused AI modeling approach described here takes into account two different materials functions or targets $Y_1$ and $Y_2$. 

In the first step, SGD is applied to identify descriptions of SGs of materials that exhibit a specific distribution of $Y_1$ values, such as a narrow distribution or a distribution concentrated in high values. SGD generates a set of inequalities describing the SG.

In the second step, SISSO is used to quantitatively model $Y_2$ for the subset of materials identified by SGD in the first step. SISSO generates a regression model describing $Y_2$. 


The outcome of the AI analysis is a SGD model that indicates whether a given material is associated with a desired distribution of $Y_1$. This classification is then subsequently quantified by a SISSO analysis which predicts the second target for the materials that were indentified by SGD. The SISSO models for do not attempt to describe all materials simultaneously, but they focus on the situations of interest that were first identified by SGD. Therefore, in this focused AI approach, SGD identifies a description of a specific data space in which the SISSO model is trained.

The focused AI approach is general and can be applied to many materials properties or functions. In this tutorial, it will be demonstrated in the context of heterogeneous catalyis.

## Application: Modeling the Performance of Pd-Based Alloys Applied in the Selective Hydrogenation of Concentrated Acetylene Streams

Heterogeneous catalysis is a key technology for enabling the efficient and sustainable production of chemicals and fuels. Chemical reactions catalyzed by solids are, nevertheless, governed by a concerted and intricate interplay of multiple processes at different scales. The complex bond-breaking and -forming reaction networks on the catalyst surface, the dynamic restructuring of the material under reaction conditions, and the transport of reactants and products to/from the catalytic surface are some of these processes. In order to model the intricacy of heterogeneous catalysis, we will combine experimental and theoretical data and identify correlations that bridge the complexity between descriptive design parameters encoding the underlying processes, and catalyst performance. Thus, we accelerate the discovery of improved, new materials while providing inisghts on the most relevant underlying processes. 

The focused AI approach will be demonstrated to model the measured time-on-stream-dependent reactivity of palladium-based bimetallic catalysts. These materials are synthesized via mechanochemistry and applied in the selective hydrogenation of concentrated acetylene streams under industrially relevant pressures, resulting from a hypothetical electric plasma-assisted methane-to-ethylene process. Unlike the well-established hydrogenation of diluted acetylene streams of naphtha steam cracking, the hydrogenation of concentrated acetylene streams remains largely underexplored due to the harsh reaction conditions and explosive nature of acetylene. This precludes operando characterization or atomistic simulations to investigate catalyst time-on-stream behavior under realistic conditions.

The first target performance of the AI approach is the acetylene conversion. The conversion reflects the catalyst activity, i.e., its ability to transform the reactant molecule into products. Thus, high acetylene conversion values are desirable. 

The second target performance is the selectivity. The selectivity reflects the relative amount of a specific product formed from the initial reactant. Three different selectivities are considered, towards the products ethylene, ethane, and C$_4$ products.  In the selective acetylene hydrogenation, the desired product is ethylene. Thus, high selectivity values for ethylene are desirable. 

The focused AI approach first uses SGD to identify descriptions of materials and reaction condition resulting in noticeable acetylene conversion. Then, it models the time-dependent selectivity focused on high acetylene conversion via the SISSO approach. 

# 2. Import Modules and Dataset

In [ ]:
# We start by importing the required modules
import os
import pandas as pd
import numpy as np
import json
import re
import shutil
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from sklearn.model_selection import KFold
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import dataset from csv file
df=pd.read_csv('./data/performance_data.csv')

# The analysis focuses on the intial reaction times
# Data points associated to nonphysical selectivity values are excluded
df_initial=df.loc[(df['time (min)'] < 406)
                 & (df['S_ethane'] > -0.1)
                 & (df['S_ethane'] < 2.1)
                 & (df['S_ethylene'] < 1)
                 & (df['S_ethylene'] > -1.2)]

# The dataset contains 12 materials measured at three temperatures (50, 100, and 150 C) 
# and multiple times on stream. In total, there are 1076 data points.
print(len(df_initial))
df_initial.head(n=10)

## Analysis of Target Performances

In [ ]:
# The analysis focuses on the target performances: acetylene conversion,
# ethylene selectivity, ethane selectivity, and C4 selectivity
performance_targets =['X_acetylene',
                     'S_ethylene',
                     'S_ethane',
                     'S_C4']

In [ ]:
# The acetyelene conversion and ethylene selectivity are plotted in the following figure
# Define catalyst names and captions
catalyst_list = [f'PdAg_1_{i}' for i in [9, 5, 1]] + \
                [f'PdAu_1_{i}' for i in [9, 5, 1]] + \
                [f'PdCu_1_{i}' for i in [9, 5, 1]] + ['Ag', 'Au', 'Cu']

catalyst_list_caption = [r'Pd$_1$Ag$_9$', r'Pd$_1$Ag$_5$', r'Pd$_1$Ag$_1$',
                         r'Pd$_1$Au$_9$', r'Pd$_1$Au$_5$', r'Pd$_1$Au$_1$',
                         r'Pd$_1$Cu$_9$', r'Pd$_1$Cu$_5$', r'Pd$_1$Cu$_1$',
                         'Ag', 'Au', 'Cu']

# Predefine temperatures and storage containers
temperatures = [50, 100, 150]
sel_matrices, act_matrices, masks = {}, {}, {}

# Extract data for each catalyst and temperature
for T in temperatures:
    sel_matrices[T], act_matrices[T] = [], []
    for cat in catalyst_list:
        data = df_initial[df_initial['material_Temperature'] == f"{cat}_{T}"]
        sel = list(data['S_ethylene'].iloc[::3])
        act = list(data['X_acetylene'].iloc[::3])
        sel_matrices[T].append(sel)
        act_matrices[T].append(act)
    
    # Convert to numpy arrays
    sel_np = np.array(sel_matrices[T])
    act_np = np.array(act_matrices[T])
    
    # Mask values where conversion is below 99%
    masks[T] = np.ma.masked_where(act_np <= 0.99, sel_np)

# Plotting 
plt.rcParams.update({'font.size': 14})
fig, axes = plt.subplots(1, 3, figsize=(12, 8), constrained_layout=True)

# Shared colormap
cmap = cm.get_cmap('viridis').copy()
cmap.set_bad(color='lightgrey')

# Shared x-ticks from any dataset
x_ticks = list(df_initial[df_initial['material_Temperature'] == f"{catalyst_list[0]}_{temperatures[0]}"]['time (min)'].iloc[::3])

# Create subplots for each temperature
for i, T in enumerate(temperatures):
    ax = axes[i]
    im = ax.imshow(masks[T], cmap=cmap, vmin=-1, vmax=0.75)

    # Axis labels and ticks
    ax.set_xlabel(r'$t_{\mathrm{OS}}$ (min)')
    ax.set_xticks(range(len(x_ticks)))
    ax.set_xticklabels(x_ticks, rotation=90)

    if i == 0:
        ax.set_yticks(range(len(catalyst_list_caption)))
        ax.set_yticklabels(catalyst_list_caption)
    else:
        ax.set_yticks([])

    # Horizontal lines to separate metal groups
    for y in range(3, masks[T].shape[0], 3):
        ax.axhline(y - 0.5, color='white', linestyle='--', linewidth=1.8)

    ax.set_title(fr'$T_{{\mathrm{{oven}}}}={T} \degree$ C')

# Shared colorbar
fig.colorbar(im, ax=axes, shrink=0.475, label=r'$S_{\mathrm{C_2H_4}}$ at $X_{\mathrm{C_2H_2}} \geq 0.99$')

## Analysis of Candidate Descriptive Parameters (Primary Features)

As candidate descriptive parameters (primary features) characterizing the materials and reaction conditions, we collected four parameters from the experimental characterization of the materials by energy dispersive X-ray analysis in a scanning electron microscope (EDX-SEM), high-resolution transmission-electron microscopy (TEM), and N$_2$ physisorption, such as the mean particle diameter ($D_\mu$) and the specific surface area ($s_{\mathrm{BET}}$).
Additionally, we included three experimental elemental (free-atom) properties and three experimental bulk properties as parameters reflecting the chemistry of the bulk of the metal NPs, such as the ionization potential ($IP$) and the closest interatomic distance ($d_{\mathrm{closest}}$). Finally, we utilized nine parameters reflecting the properties of the surfaces of the metal NPs and the interaction of carbon and hydrogen with the surface and subsurface. These parameters were calculated by density functional theory under the generalized gradient approximation (DFT-GGA) on low-index model surfaces. Examples of such parameters are the energy of the $d$-band center($\epsilon_d$) and the binding energy of subsurface hydrogen and carbon ($E_{\mathrm{b,H}}^{\mathrm{sub}}$ and $E_{\mathrm{b,C}}^{\mathrm{sub}}$, respectively). The elemental, bulk, and surface-related properties were converted into materials-specific parameters by taking the composition average, indicated by the bar in $\overline{\phi}$, where $\phi$ is an elemental, bulk, or surface parameter. Finally, $t_{\mathrm{OS}}$ and $T_{\mathrm{oven}}$ were used as parameters related to the applied reaction conditions. In total, 21 candidate descriptive parameters were collected.

In [ ]:
# We collect the values of the candidate descriptive parameters 
# plot the distribution of their values, and analyze the correlations between them

reaction_parameters=['time (min)','T_reactor (C)']

materials_parameters=['total_metal_loading', 
                      'particle_diameter (nm)',
                      'std_dev (nm)', 
                      'surface_area (m^2/g)',
                      'av_IP (eV)', 
                      'av_EA (eV)', 
                      'av_PE',
                      'av_bulk_int_dist (A)', 
                      'av_E_coh (eV/atom)', 
                      'av_B_0 (Gpa)', 
                      'av_mu_C_critical_surf (eV)',
                      'av_mu_C_critical_subsurf (eV)', 
                      'av_E_b_C_subsurf (eV)', 
                      'av_E_def_subsurf (eV)',
                      'av_delta_M1_M2_subsurf', 
                      'av_BE_H_surf (eV)', 
                      'av_W_change_H (eV)',
                      'av_d_band_center_surf (eV)', 
                      'av_BE_H_subsurf (eV)']

materials_parameters_labels=['$w_{\mathrm{metal}}$' , 
                             '$D_\mu$', 
                             '$D_\sigma$',
                             '$s_{\mathrm{BET}}$', 
                             '$\overline{IP}$', 
                             '$\overline{EA}$', 
                             '$\overline{EN}$',
                             '$\overline{d_{\mathrm{closest}}}$', 
                             '$\overline{E_{\mathrm{coh}}}$', 
                             '$\overline{B_{\mathrm{0}}}$',
                             '$\overline{\mu_{\mathrm{C}}^{\mathrm{surf}}}$',
                             '$\overline{\mu_{\mathrm{C}}^{\mathrm{sub}}}$',
                             '$\overline{E_{\mathrm{b,C}}^{\mathrm{sub}}}$', 
                             '$\overline{E_{\mathrm{d,C}}^{\mathrm{sub}}}$',
                             '$\overline{\delta_{\mathrm{C}}^{\mathrm{sub}}}$', 
                             '$\overline{E_{\mathrm{b,H}}^{\mathrm{surf}}}$', 
                             '$\overline{\Delta W_{\mathrm{H}}^{\mathrm{surf}}}$',
                             '$\overline{\epsilon_d}$',
                             '$\overline{E_{\mathrm{b,H}}^{\mathrm{sub}}}$']

features=reaction_parameters+materials_parameters

df_materials_parameters = df_initial[materials_parameters].copy()

In [ ]:
# Create histogram grid
fig, axes = plt.subplots(5, 4, figsize=(16, 14))
axes = axes.flatten()

# Loop over each parameter and its label
for i, (param, label) in enumerate(zip(materials_parameters, materials_parameters_labels)):
    ax = axes[i]
    ax.hist(df_materials_parameters[param], bins=20, color='k')
    ax.set_title(label)
    #ax.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)

# Hide any unused subplots
for j in range(len(materials_parameters), len(axes)):
    axes[j].axis('off')

# Adjust layout
fig.suptitle('Distribution of Material Parameters', fontsize=18)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
plt.rcParams.update({'font.size': 10})
fig,ax1 = plt.subplots(1,1, constrained_layout=True, figsize=(5,5))


matrix = np.triu(df_materials_parameters)
pearson_correlation = df_materials_parameters.corr(method='pearson')
sns.heatmap(pearson_correlation, 
            xticklabels=materials_parameters_labels,
            yticklabels=materials_parameters_labels,
            cmap='RdBu_r',
            annot=False,
            linewidth=0.5,
            vmin=-1, vmax=1)

# 3. SGD Identification of Descriptions of Materials and Reaction Conditions Resulting in Noticeable Acetylene Conversion

In [ ]:
# Specify columns to extract (index, target, and features to be used in SGD)
columns_to_extract =['material_Temperature','material_Temperature_time']+performance_targets+features

# Extract the specified columns
df_SGD = df_initial[columns_to_extract].copy().set_index('material_Temperature_time')

# Rename columns by removing the units (in parentheses)
df_SGD.columns = [re.sub(r'\s*\(.*?\)', '', col).strip() for col in df_SGD.columns]

In [ ]:
# Function to create input files for SGD
def write_input_SGD(path, df, id_job, n_cutoffs, algo, dev, n_res, n_seeds, target_key, weight):
    """
    creates the two input files necessary to run SGD with realkd:
    i) a .json file with calculation details, named "id_job.json", and
    ii) a .xarf file with the data set, named "id_job.xarf".
    function arguments: path(str): path to the folder where the files 
                                   will be written
               df(data frame): data set containing the values for the 
                               candidate descriptive parameters and for
                               the target for all materials
               id_job(str): job name
               n_cutoffs(int): number of cutoffs to be used in k-Means
                               clustering to generate the propositions
               algo(str): SG search algorithm:
                          PMM_SAMPLER 
                          EMM_SAMPLER
                          EXCEPTIONAL_SUBGROUP_BESTFIRST_BRANCHANDBOUND
                          
                          PMM_SAMPLER uses (std(SG)-std(P))/std(P) as utility function
                          whereas EMM_SAMPLER/EXCEPTIONAL_SUBGROUP_BESTFIRST_BRANCHANDBOUND 
                          use the function specified in dev
               
               dev(str): deviation measure when using EMM_SAMPLER: 
                         cumulative_jensen_shannon_divergence
                         normalized_positive_mean_shift
                         normalized_negative_mean_shift
                         normalized_positive_median_shift
                         normalized_negative_median_shift
                         
               n_res(int): number of results, i.e., number of top-ranked
                           SGs to display
               n_seeds(int): number of seeds to use for the SG search
               target_key(str): label of the variable to be used as target quantity in SGD
    """
    df.to_csv(path+'/'+id_job+'.csv')
    with open(path+'/'+id_job+'.csv', 'r') as file_in:
        data = file_in.read().splitlines(True)
        
    file_out = open(path+'/'+id_job+'.xarf', 'w')
    file_out.write('@relation '+id_job+'\n')
    file_out.write('@attribute materials name\n')
    for variable in list(df.columns):
        file_out.write('@attribute '+variable+' numeric\n')
    file_out.write("@data\n")
    file_out.close()

    with open(path+'/'+id_job+'.xarf', 'a') as file_out:
        file_out.writelines(data[1:])
        file_out.close()
    
    input_file = {}
    input_file = {"type" : "productWorkScheme",
                  "id" : id_job,
                  "workspaces" : [ {
                                "type" : "workspaceFromXarf",
                                "id" : id_job,
                                "datafile" : id_job+".xarf",
                                "propScheme": {"type": "standardPropScheme",
                                                "defaultMetricRule": {"type": "kmeansPropRule",
                                                                       "numberOfCutoffs": n_cutoffs,
                                                                       "maxNumberOfIterations": 1000}}} ],
                    "computations" : [ {
                                "type" : "legacyComputation",
                                "id" : "subgroup_analysis",
                                "algorithm" : algo,
                                "parameters" : {
                                    "dev_measure": dev,
                                    "attr_filter" : "[]",
                                    "cov_weight" : weight,
                                    "num_res" : n_res,
                                    "num_seeds" : n_seeds,
                                    "targets" : "["+target_key+"]"
                                             }
                  }],
                  "computationTimeLimit" : 3600000
                     }
    with open(path+'/'+id_job+'.json','w') as outfile:
        json.dump(input_file, outfile, indent=4)

## Attention ⚠️

We will use the standard-deviation-reduction utility function, defined by: 

\begin{equation}
 u(SG,\widetilde{P}) = \frac{std(\widetilde{P})-std(SG)}{std(\widetilde{P})}, 
\end{equation} 

where $std(SG)$ and $std(\widetilde{P})$ are the standard deviation of the distributions of the target in the SG and in the whole data set, respectively. By using this utility function, we favor the selection of SGs that present narrow distribution of target values.

In [ ]:
# Specify a name for the SGD job
job_id='SGD_results_publication'

# Speficy SGD parameters
# Number of thresholds (k in k-means clustering)
n_clusters=20

#Number of seeds used to initialize the SGD search algorithm (opportunistic pruning)
n_seeds=50000

#Number of SGD solutions to be printed
n_results=5000

#Check if old files exist for the specified job_id and remove them
#dirpath = os.path.join('./output/', job_id)
#if os.path.exists(dirpath) and os.path.isdir(dirpath):
#    shutil.rmtree(dirpath)

# Write input files
# The utility function "standard deviation reduction" is chosen ("PMM_SAMPLER")
#write_input_SGD('./', 
#            df_SGD.drop(['material_Temperature', 'time', 'S_ethylene', 'S_ethane','S_C4'], axis=1), 
#            job_id, 
#            n_clusters,
#            'PMM_SAMPLER',
#            '',
#            n_results, 
#            n_seeds,
#            columns_to_extract[2],
#            1.0)
# Run SGD  
#os.system('java -jar realkd-0.7.2-jar-with-dependencies.jar '+job_id+'.json')

In [ ]:
# Function to analyze the results of SGD
def analyze_output_SGD(file_results):
    
    list_coverages=[]
    list_utility_function=[]
    list_quality_function=[]
    list_target_mean=[]
    list_constraints=[]
    
    with open(file_results) as json_file:
        data = json.load(json_file)
        for index in range(len(data)):
            coverage=data[index].get('measurements')[0].get('value')
            utility_function=data[index].get('measurements')[1].get('value')
            quality_function=coverage*utility_function
            target_mean=data[index].get('descriptor').get('targetLocalModel').get('means')
            list_attributes=data[index].get('descriptor').get('selector').get('attributes')
            list_operators=[]
            list_cutoffs=[]
            constraints=[]
            for i in list(range(0,len(list_attributes))):
                list_operators.append(data[index].get('descriptor').get('selector').get('constraints')[i].get('type'))
                list_cutoffs.append(round(data[index].get('descriptor').get('selector').get('constraints')[i].get('value'),4))

            list_operators = [op.replace('lessOrEquals', '<=') for op in list_operators]
            list_operators = [op.replace('greaterOrEquals', '>=') for op in list_operators]
            list_operators = [op.replace('lessThan', '<') for op in list_operators]
            list_operators = [op.replace('greaterThan', '>') for op in list_operators]
    
            for i in list(range(0,len(list_attributes))):
                if i == 0:
                    constraints=list_attributes[0]+list_operators[0]+str(list_cutoffs[0])
                else:
                    constraints=constraints+' & '+list_attributes[i]+list_operators[i]+str(list_cutoffs[i])
            list_coverages.append(coverage)
            list_utility_function.append(utility_function)
            list_quality_function.append(quality_function)
            list_target_mean.append(*target_mean)
            list_constraints.append(constraints)
            
            
    df = pd.DataFrame(list(zip(list_coverages,
                               list_utility_function,
                               list_quality_function,
                              list_target_mean,
                               list_constraints)), 
                      columns =['coverage','utility','quality','target_mean','constraints'])
    return(df)

In [ ]:
# Path to the file with the SGD results
file_results='./output/'+job_id+'/'+os.listdir('./output/'+job_id+'/')[0]+'/results/'+job_id+'_subgroup_analysis.json'

# The results are stored in a dataframe that shows a list of SGs ranked by their quality-function values 
df_SGD_results=analyze_output_SGD(file_results)
df_SGD_results.head(n=10)

In [ ]:
# Collect the rules associated to the SG presenting the highest value of quality function
df_SGD_results.iloc[0]['constraints']

In [ ]:
# Plot the distribution of target performance metrics in the entire dataset
# and in the identified SG
plt.rcParams.update({'font.size': 15})
fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, constrained_layout=True, figsize=(6,6))

# Apply the SG rules (constraints) to the dataset
df_SGD_selected=df_SGD.query(df_SGD_results['constraints'][0])

color_entire_dataset='black'
color_SG='red'
n_bins=20

ax1.hist(df_initial['X_acetylene'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
ax1.hist(df_SGD_selected['X_acetylene'], color=color_SG, bins=n_bins, rwidth=0.8)

axins = inset_axes(ax1, width="65%", height="35%",loc=2)
axins.hist(df_initial['X_acetylene'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
axins.hist(df_SGD_selected['X_acetylene'], color=color_SG, bins=n_bins, rwidth=0.8)
axins.set_ylim(0,40)

ax2.hist(df_initial['S_ethylene'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
ax2.hist(df_SGD_selected['S_ethylene'], color=color_SG, bins=n_bins, rwidth=0.8)

ax3.hist(df_initial['S_ethane'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
ax3.hist(df_SGD_selected['S_ethane'], color=color_SG, bins=n_bins, rwidth=0.8)

ax4.hist(df_initial['S_C4'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
ax4.hist(df_SGD_selected['S_C4'], color=color_SG, bins=n_bins, rwidth=0.8)

ax1.set_ylabel('Counts')
ax2.set_ylabel('Counts')
ax3.set_ylabel('Counts')
ax4.set_ylabel('Counts')

ax1.set_xlabel('$X_{\mathrm{acetylene}}$')
ax2.set_xlabel('$S_{\mathrm{ethylene}}$')
ax3.set_xlabel('$S_{\mathrm{ethane}}$')
ax4.set_xlabel('$S_{\mathrm{C4}}$')

In [ ]:
# For the SG identified in the publication,
# we plot the 12 materials in the coordinates of the two materials-related parameters appearing in the rules
fig, (ax1) = plt.subplots(1,1, constrained_layout=True, figsize=(4,4))
marker_size=30
ax1.vlines(4.6685,2.5659,3.05,color='red',linestyle='dashed')
ax1.hlines(2.5659,4.6685,9,color='red',linestyle='dashed')
ax1.fill_between([4.6685,9], 2.5659, 3.05, color='mistyrose')

ax1.scatter(df_SGD['av_E_b_C_subsurf'],df_SGD['av_bulk_int_dist'],c=color_entire_dataset,s=marker_size)
ax1.scatter(df_SGD_selected['av_E_b_C_subsurf'],df_SGD_selected['av_bulk_int_dist'],c=color_SG,s=marker_size)

ax1.set_ylim(2.5,3.05)
ax1.set_xlim(4,7.25)
ax1.arrow(4.6685, 2.7, 0.5, 0, head_width=0.015, head_length=0.2, color=color_SG)
ax1.arrow(5.8, 2.5659, 0, 0.06, head_width=0.10, head_length=0.03, color=color_SG)

ax1.set_xlabel('$\\widebar{E_{\mathrm{b,C}}^{\mathrm{sub}}}$ (eV)')  
ax1.set_ylabel('$\\widebar{d_{\mathrm{closest}}}$ ($\mathrm{\AA}$)') 

## Task ⚠️
Try the positive-mean-shift utility function, defined as:

\begin{equation}
    u(SG,\widetilde{P})=\frac{\bar{Y}(SG)-\bar{Y}(\widetilde{P})}{Y_{\mathrm{max}}(\widetilde{P})-\bar{Y}(\widetilde{P})}.
\end{equation}

and the negative-mean-shift utility function, defined as:

\begin{equation}
    u(SG,\widetilde{P})=\frac{-(\bar{Y}(SG)-\bar{Y}(\widetilde{P}))}{Y_{\mathrm{max}}(\widetilde{P})-\bar{Y}(\widetilde{P})}.
\end{equation}

In the equations above, $\bar{Y}(SG)$ and $\bar{Y}(\widetilde{P})$ are the mean values of the distribution of the target in the SG and in the entire dataset, and $Y_{\mathrm{max}}(\widetilde{P})$ is the maximum value that the target assumes in the dataset.

**Question:** How different are the SGs identified with the different utility functions?

In [ ]:
# Specify a name for the SGD job
job_id='SGD_positive_mean'

# Speficy SGD parameters
# Number of thresholds (k in k-means clustering)
n_clusters=20

#Number of seeds used to initialize the SGD search algorithm (opportunistic pruning)
n_seeds=50000

#Number of SGD solutions to be printed
n_results=5000

#Check if old files exist for the specified job_id and remove them
dirpath = os.path.join('./output/', job_id)
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)

# Write input files
# The utility function "standard deviation reduction" is chosen ("PMM_SAMPLER")
write_input_SGD('./', 
            df_SGD.drop(['material_Temperature', 'time', 'S_ethylene', 'S_ethane','S_C4'], axis=1), 
            job_id, 
            n_clusters,
            'EMM_SAMPLER',
            'normalized_positive_mean_shift',
            n_results, 
            n_seeds,
            columns_to_extract[2],
            1.0)
# Run SGD  
os.system('java -jar realkd-0.7.2-jar-with-dependencies.jar '+job_id+'.json')

In [ ]:
# Path to the file with the SGD results
file_results='./output/'+job_id+'/'+os.listdir('./output/'+job_id+'/')[0]+'/results/'+job_id+'_subgroup_analysis.json'

# The results are stored in a dataframe that shows a list of SGs ranked by their quality-function values 
df_SGD_results=analyze_output_SGD(file_results)

# Plot the distribution of target performance metrics in the entire dataset
# and in the identified SG
plt.rcParams.update({'font.size': 15})
fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, constrained_layout=True, figsize=(6,6))

# Apply the SG rules (constraints) to the dataset
df_SGD_selected=df_SGD.query(df_SGD_results['constraints'][0])

color_entire_dataset='black'
color_SG='red'
n_bins=20

ax1.hist(df_initial['X_acetylene'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
ax1.hist(df_SGD_selected['X_acetylene'], color=color_SG, bins=n_bins, rwidth=0.8)

axins = inset_axes(ax1, width="65%", height="35%",loc=2)
axins.hist(df_initial['X_acetylene'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
axins.hist(df_SGD_selected['X_acetylene'], color=color_SG, bins=n_bins, rwidth=0.8)
axins.set_ylim(0,40)

ax2.hist(df_initial['S_ethylene'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
ax2.hist(df_SGD_selected['S_ethylene'], color=color_SG, bins=n_bins, rwidth=0.8)

ax3.hist(df_initial['S_ethane'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
ax3.hist(df_SGD_selected['S_ethane'], color=color_SG, bins=n_bins, rwidth=0.8)

ax4.hist(df_initial['S_C4'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
ax4.hist(df_SGD_selected['S_C4'], color=color_SG, bins=n_bins, rwidth=0.8)

ax1.set_ylabel('Counts')
ax2.set_ylabel('Counts')
ax3.set_ylabel('Counts')
ax4.set_ylabel('Counts')

ax1.set_xlabel('$X_{\mathrm{acetylene}}$')
ax2.set_xlabel('$S_{\mathrm{ethylene}}$')
ax3.set_xlabel('$S_{\mathrm{ethane}}$')
ax4.set_xlabel('$S_{\mathrm{C4}}$')

In [ ]:
# Specify a name for the SGD job
job_id='SGD_negative_mean'

# Speficy SGD parameters
# Number of thresholds (k in k-means clustering)
n_clusters=20

#Number of seeds used to initialize the SGD search algorithm (opportunistic pruning)
n_seeds=50000

#Number of SGD solutions to be printed
n_results=5000

#Check if old files exist for the specified job_id and remove them
dirpath = os.path.join('./output/', job_id)
if os.path.exists(dirpath) and os.path.isdir(dirpath):
    shutil.rmtree(dirpath)

# Write input files
# The utility function "standard deviation reduction" is chosen ("PMM_SAMPLER")
write_input_SGD('./', 
            df_SGD.drop(['material_Temperature', 'time', 'S_ethylene', 'S_ethane','S_C4'], axis=1), 
            job_id, 
            n_clusters,
            'EMM_SAMPLER',
            'normalized_negative_mean_shift',
            n_results, 
            n_seeds,
            columns_to_extract[2],
            1.0)
# Run SGD  
os.system('java -jar realkd-0.7.2-jar-with-dependencies.jar '+job_id+'.json')

In [ ]:
# Path to the file with the SGD results
file_results='./output/'+job_id+'/'+os.listdir('./output/'+job_id+'/')[0]+'/results/'+job_id+'_subgroup_analysis.json'

# The results are stored in a dataframe that shows a list of SGs ranked by their quality-function values 
df_SGD_results=analyze_output_SGD(file_results)

# Plot the distribution of target performance metrics in the entire dataset
# and in the identified SG
plt.rcParams.update({'font.size': 15})
fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, constrained_layout=True, figsize=(6,6))

# Apply the SG rules (constraints) to the dataset
df_SGD_selected=df_SGD.query(df_SGD_results['constraints'][0])

color_entire_dataset='black'
color_SG='red'
n_bins=20

ax1.hist(df_initial['X_acetylene'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
ax1.hist(df_SGD_selected['X_acetylene'], color=color_SG, bins=n_bins, rwidth=0.8)

axins = inset_axes(ax1, width="65%", height="35%",loc=2)
axins.hist(df_initial['X_acetylene'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
axins.hist(df_SGD_selected['X_acetylene'], color=color_SG, bins=n_bins, rwidth=0.8)
axins.set_ylim(0,40)

ax2.hist(df_initial['S_ethylene'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
ax2.hist(df_SGD_selected['S_ethylene'], color=color_SG, bins=n_bins, rwidth=0.8)

ax3.hist(df_initial['S_ethane'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
ax3.hist(df_SGD_selected['S_ethane'], color=color_SG, bins=n_bins, rwidth=0.8)

ax4.hist(df_initial['S_C4'], color=color_entire_dataset, bins=n_bins, rwidth=0.8)
ax4.hist(df_SGD_selected['S_C4'], color=color_SG, bins=n_bins, rwidth=0.8)

ax1.set_ylabel('Counts')
ax2.set_ylabel('Counts')
ax3.set_ylabel('Counts')
ax4.set_ylabel('Counts')

ax1.set_xlabel('$X_{\mathrm{acetylene}}$')
ax2.set_xlabel('$S_{\mathrm{ethylene}}$')
ax3.set_xlabel('$S_{\mathrm{ethane}}$')
ax4.set_xlabel('$S_{\mathrm{C4}}$')

## Extra Task ⚠️
Use the cumulative Jensen-Shannon divergence utility function.

# 4. Training SISSO Models for Selectivity 

In [ ]:
# We first visualize the selectivity values as a function of time on stream for the datapoints selected in the SG

# Path to the file with the SGD results of the publication
job_id='SGD_results_publication'
file_results='./output/'+job_id+'/'+os.listdir('./output/'+job_id+'/')[0]+'/results/'+job_id+'_subgroup_analysis.json'
df_SGD_results=analyze_output_SGD(file_results)
df_SGD_selected=df_SGD.query(df_SGD_results['constraints'][0])

# Configure global plot font size
plt.rcParams.update({'font.size': 15})

# Create grid of subplots (for three products at two temperatures)
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(12, 9), constrained_layout=True)

# Define visual properties for catalysts 
colors = ['grey'] * 3 + ['darkorange'] * 3 + ['brown'] * 3
markers = ['s', '>', 'o'] * 3
marker_size = 30

# Loop over each of the 9 catalysts
for i in range(9):    
    # Get time-series data for current catalyst at 100°C and 150°C
    cat_100 = df_SGD_selected.loc[df_SGD_selected['material_Temperature'] == catalyst_list[i] + '_100']
    cat_150 = df_SGD_selected.loc[df_SGD_selected['material_Temperature'] == catalyst_list[i] + '_150']
    
    # Scatter plot for S_ethylene
    ax1.scatter(cat_100['time'], cat_100['S_ethylene'], color=colors[i], s=marker_size, marker=markers[i])
    ax2.scatter(cat_150['time'], cat_150['S_ethylene'], color=colors[i], s=marker_size, marker=markers[i])
    
    # Scatter plot for S_ethane
    ax3.scatter(cat_100['time'], cat_100['S_ethane'], color=colors[i], s=marker_size, marker=markers[i])
    ax4.scatter(cat_150['time'], cat_150['S_ethane'], color=colors[i], s=marker_size, marker=markers[i])
    
    # Scatter plot for S_C4
    ax5.scatter(cat_100['time'], cat_100['S_C4'], color=colors[i], s=marker_size, marker=markers[i])
    ax6.scatter(cat_150['time'], cat_150['S_C4'], color=colors[i], s=marker_size, marker=markers[i])

# Set axes limits
for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
    ax.set_xlim(0, 406)

ax1.set_ylim(-1.1, 1.5)
ax2.set_ylim(-1.1, 1.5)
ax3.set_ylim(0, 2.1)
ax4.set_ylim(0, 2.1)
ax5.set_ylim(0, 0.2)
ax6.set_ylim(0, 0.2)

# Set axis labels
ax1.set_ylabel('$S_{\mathrm{C_2H_4}}$')
ax3.set_ylabel('$S_{\mathrm{C_2H_6}}$')
ax5.set_ylabel('$S_{\mathrm{C4}}$')
ax5.set_xlabel('$t_{\mathrm{OS}}$ (min)')
ax6.set_xlabel('$t_{\mathrm{OS}}$ (min)')

# Tweak tick labels visibility for cleaner layout
for ax in [ax1, ax2, ax3, ax4]:
    ax.set_xticklabels([])
for ax in [ax2, ax4, ax6]:
    ax.set_yticklabels([])

# Add subplot titles for temperature conditions
ax1.set_title('$T_{\mathrm{oven}} = 100\,^\circ\mathrm{C}$')
ax2.set_title('$T_{\mathrm{oven}} = 150\,^\circ\mathrm{C}$')

# Add horizontal reference line at y=0 to the top row
ax1.axhline(0, color='k', linewidth=0.5)
ax2.axhline(0, color='k', linewidth=0.5)

# Add legend for catalysts (only once)
ax2.legend(
    labels=catalyst_list_caption[:9],
    loc=(1.04, 0.05),
    frameon=True,
    ncol=1,
    fontsize=10)

In [ ]:
# A dataset with the training data for SISSO is created based on the SG rules identified in the publication
# It contains 539 data points, ca. 50% of the original dataset 
df_SISSO=df_initial.loc[(df_initial['T_reactor (C)'] > 75)
                      & (df_initial['av_bulk_int_dist (A)'] >=2.5659)
                      & (df_initial['av_E_b_C_subsurf (eV)'] >= 4.6685)]
len(df_SISSO)

## Attention ⚠️

SISSO has two hyperparameters, the rung $q$ and the model dimensionality $D$. These hyperparameters are not learned during training and thus and thus need to be determined by evaluating the model performance in unseen data. For this purpose, a nested 5-fold cross-validation (CV) scheme is used: 

<img style="float: center;" src="data/nested_CV.png" width=650>


The outer loop of the CV scheme is used to evaluate prediction errors. These are denoted *test errors* in the figure. The inner loop is used for evaluating the optimal model hyperparameters. The optimal set of hyperparameters is considered the one providing the lowest errors in the unseen data, denoted *validation errors*. In particular, we use the root mean squared error (RMSE) averaged over the five validation sets as our performance metric. Here, we consider $q=1,2$ and $D=1,2,3,4,5$. 

The sample indices correspoding to the nested 5-fold procedure are obtained as follows.

In [ ]:
kf_out=KFold(n_splits=5, random_state=10, shuffle=True)

indices_train_out=[]
indices_test=[]

for train_index,test_index in kf_out.split(df_SISSO):
    indices_train_out.append(train_index)
    indices_test.append(test_index)

kf_in=KFold(n_splits=5, random_state=10, shuffle=True)

indices_train_in=[]
indices_validation=[]

for train_index,test_index in kf_in.split(df_SISSO.iloc[indices_train_out[0],:]):
    indices_train_in.append(train_index)
    indices_validation.append(test_index)

In [ ]:
# Function to create input files for SISSO
def write_input_SISSO(train,rung,dim,n_sis,n_res,ops,leave_out_ind,path,calculation_type):
    """Creates the directory and writes the input files (sisso.json and train.dat) for SISSO++
    
    Args:
        train (pd.DataFrame): dataframe containing the data set
        rung (int): number of iterations for operator application (e.g. 1, 2 or 3)
        dim (int): descriptor dimension
        n_sis (int): size of the SIS-selected feature spaces
        n_res (int): number of residuals
        ops (list): list of mathematical operators (str)
        leave_out_ind (list): index of the materials to be left out from training, i.e., test materials
        path (str): path of the directory to be created 
        calculation_type (str): type of calculation (e.g., regression or log_regression)
    """
    rung_store = max(0, rung - 1)
    rung_gen = 0 if rung < 3 else 1
        
    os.mkdir(path)
    train.to_csv(f"{path}/train.dat")
    prop_key=train.columns.values[0].split()[0]
    input_file = {
        'desc_dim':dim,
        'n_sis_select':n_sis,
        'max_rung':rung,
        'n_rung_store':rung_store,
        'n_residual':n_res,
        'min_abs_feat_val': 1e-6,
        'max_abs_feat_val': 1e6,
        'data_file': 'train.dat',
        'property_key': prop_key,
        'n_rung_generate': rung_gen,
        'leave_out_inds': leave_out_ind,
        'n_models_store': 10,
        'opset': ops, 
        "param_opset": [
        "exp",
        "log"], 
        'global_param_opt': 'True',
        'calc_type': calculation_type,
    }
    with open(f"{path}/sisso.json",'w') as outfile:
        json.dump(input_file, outfile, indent=4)

In [ ]:
# Speficy SISSO parameters

# Set of operators utilized to create the analytical expressions
operator_set = ["add", 
                "sub", 
                "abs_diff", 
                "mult", 
                "div", 
                "inv", 
                "exp", 
                "sq", 
                "cb", 
                "sqrt", 
                "cbrt", 
                "log", 
                "abs", 
                "six_pow",
                "neg_exp"] 

# Number of residuals
n_res=5

#Number of model dimensions
dimension=5

#Size of subspaces of features selected by SIS
n_sis=50

In [ ]:
# We create directories and write SISSO input files associated to the nested cross-validation scheme
# We also train SISSO models utilizing the entire dataset of 539 data points 

#os.mkdir('./output/full_dataset')
#for fold in [0,1,2,3,4]:
#    os.mkdir('./output/fold_'+str(fold))
        
#for target in performance_targets[1:]:
#    for rung in [1,2]:
#        df_SISSO_outer_loop=df_SISSO[['material_Temperature']+[target]+['task']+features].copy().set_index('material_Temperature')
#        write_input_SISSO(df_SISSO_outer_loop,
#                        rung, dimension, n_sis, n_res, operator_set,
#                        '[]',
#                        './output/full_dataset/'+target+'_r'+str(rung),
#                       'regression')
#        for fold in [0,1,2,3,4]:
#            write_input_SISSO(df_SISSO_outer_loop,
#                        rung, dimension, n_sis, n_res, operator_set,
#                        indices_test[fold].tolist(),
#                        './output/fold_'+str(fold)+'/'+target+'_r'+str(rung),
#                       'regression')
            
#            os.mkdir('./output/fold_'+str(fold)+'/'+target+'_r'+str(rung)+'_cv')
#            for it in [0,1,2,3,4]:
#                    write_input_SISSO(df_SISSO_outer_loop.iloc[indices_train_out[fold],:],
#                                rung, dimension, n_sis, n_res, operator_set,
#                                 indices_validation[it].tolist(),
#                                 './output/fold_'+str(fold)+'/'+target+'_r'+str(rung)+'_cv/iter_'+str(it),
#                                'regression')     

In this tutorial, we will not run the SISSO algorithm, but we will rather read the output files of SISSO provided. 

In [ ]:
# Function to obtain the validation errors of SISSO models
def get_rmse_vs_dim(path,path_cv,n_iter,dim,unit_length):
    """
    reads SISSO++ output (sisso.out) and returns train and validation errors as a function of descriptor dimension
    arguments: path(str): directory containing the output files obtained with the outer-loop data set splits
               path_cv(str): directory containing the output files obtained with the inner-loop data set splits (cross-validation)
               n_iter(int): number of CV iterations
               dim(int): descriptor max dimension
               unit_length(int): lenght of the property unit
    """
    train_errors=[]
    cv_errors=[]
    avg_cv_errors=[]
    std_cv_errors=[]
    with open(path+'/sisso.out') as f:
        data = f.readlines()
        i = 0
        for line in data:
            if line.__contains__('Train'):
                train_rmse =float(line.split()[2].split(';')[0])
                train_errors.append(train_rmse)
                i += 1
    
    for it in [*range(n_iter)]:
        with open(path_cv+'/iter_'+str(it)+'/sisso.out') as f:
            data = f.readlines()
            i = 0
            for line in data:
                if line.__contains__('Train'):
                    test_rmse = float(line.split()[6+(unit_length-1)])
                    cv_errors.append([i, test_rmse])
                    i += 1
    for d in [*range(dim)]:
        a=[]
        for k in [*range(len(cv_errors))]:
            if cv_errors[k][0] == d:
                a.append(cv_errors[k][1])
        avg_cv_errors.append(np.average(a))
        std_cv_errors.append(np.std(a))
                
    return(train_errors, avg_cv_errors, std_cv_errors) 

# Function to obtain the predictions of SISSO models on the test sets
def get_test_data(path,n_iter,rung,dim,prop):
    """
    """
    test_data=pd.DataFrame()
    for it in [*range(n_iter)]:
        d=pd.read_csv(path+'fold_'+str(it)+'/'+str(prop)+'_r'+str(rung)+'/models/test_dim_'+str(dim)+'_model_0.dat', comment='#', header=None)
        test_data=pd.concat([test_data,d])
    return(test_data) 


In [ ]:
# Visualize the validation errors utilizing the output files provided in ./output/SISSO_results_publication

# Set default font size for plots
plt.rcParams.update({'font.size': 16})

# X-axis values and style definitions
x = [1, 2, 3, 4, 5]
colors = ['', 'red', 'blue']  # rung 1: red, rung 2: blue
labels = ['', '$q=1$', '$q=2$']

# Loop through each performance target
for target in performance_targets[1:]:
    # Define y-axis upper limit as a fraction of the target's std deviation
    std_of_target = np.std(df_SISSO[target]) * 0.6

    # Create 1 row x 5 columns subplot grid, one per data fold
    fig, axes = plt.subplots(1, 5, figsize=(15, 4), constrained_layout=True)

    for fold, ax in enumerate(axes):
        # Plot results for each rung (model complexity level)
        for rung in [1, 2]:
            # Build file path and load RMSE values
            path = f'./output/SISSO_results_publication/fold_{fold}/{target}_r{rung}'
            errors = get_rmse_vs_dim(path, path + '_cv', 5, 5, 1)

            # Plot with error bars
            ax.errorbar(x, errors[1], yerr=errors[2],
                        color=colors[rung],
                        marker='o',
                        mfc=colors[rung],
                        capsize=4,
                        label=labels[rung])

        # Formatting
        ax.set_xlim(0.5, 5.5)
        ax.set_xticks([1, 2, 3, 4, 5])
        ax.set_xlabel('$D$')  # D = descriptor dimension

        # Y-axis settings
        ax.set_ylim(0, std_of_target)
        if fold == 0:
            ax.set_ylabel('Cross-validation RMSE')
            ax.set_title(target)
        else:
            ax.set_yticks([])  # Remove y-ticks from all but the first plot

        # Add second Y-axis
        ax_twin = ax.twinx()
        ax_twin.set_ylim(0, 60)
        #ax_twin.set_yticks([])
        if fold == 4:
            ax_twin.set_ylabel('Cross-validation RMSE / $\sigma$ (%)')

        # Add legend only in the first plot
        if fold == 0:
            ax.legend(loc='upper right', frameon=True, fontsize=14)

## Task ⚠️

**Question:** What hyperparameters would you choose?

In [ ]:
# Visualize the test errors utilizing the output files provided in ./output/SISSO_results_publication

# Plot style settings
plt.rcParams.update({'font.size': 16})

# Optimal rung and descriptor dimension for each target, as identified based on the validation errors (above)
optimal_rungs = [2, 2, 2]
optimal_dimensions = [3, 3, 3]

# Plot customization for boxplots and violin plots
colors = ['lightblue']
boxprops = {"zorder": 100, "linewidth": 1.5, "edgecolor": "k", "facecolor": "none"}
whiskerprops = {"zorder": 100, "color": "k", "linewidth": 1.5}
capprops = {"zorder": 100, "color": "k", "linewidth": 1.5}
medianprops = {"linewidth": 1.5, "color": "orange", "zorder": 200}
meanprops = {
    "marker": "x", "markersize": 10.0,
    "markerfacecolor": "k", "markeredgecolor": "k",
    "zorder": 500
}

# Descriptor labels for each subplot
xlabels = ['$S_{\mathrm{C_2H_4}}$', '$S_{\mathrm{C_2H_6}}$', '$S_{\mathrm{C_4}}$']

# Create a single-row, 3-column plot layout
fig, axes = plt.subplots(1, 3, figsize=(12, 4), constrained_layout=True)

# Loop over the three properties to generate subplots
for i, ax in enumerate(axes):
    # Get standard deviation of the target variable (for error normalization)
    target = performance_targets[i + 1]
    std_of_target = np.std(df_SISSO[target])

    # Get prediction error data for the optimal model settings
    test_data = get_test_data('./output/SISSO_results_publication/', 5, optimal_rungs[i], optimal_dimensions[i], target)
    test_data["error"] = abs(test_data[1] - test_data[2])
    test_data["type"] = 1  # Constant x-category

    # Violin plot
    sns.violinplot(
        data=test_data, x="type", y="error",
        inner=None, scale="area", palette=colors,
        alpha=1.0, linewidth=0.0, zorder=-2, ax=ax
    )

    # Boxplot
    sns.boxplot(
        data=test_data, x="type", y="error",
        showfliers=True, showmeans=True,
        boxprops=boxprops, whiskerprops=whiskerprops,
        capprops=capprops, medianprops=medianprops,
        meanprops=meanprops, width=0.35, whis=(0.0, 95), ax=ax
    )

    # Axes formatting
    ax.set_xticklabels([''], va='top', fontsize=16)
    ax.set_ylim(0, std_of_target * 1.5)
    ax.set_xlabel(xlabels[i])
    if i == 0:
        ax.set_ylabel('Absolute Test Error')
    else:
        ax.set_ylabel('')

    # Secondary y-axis for percent-normalized view
    ax_twin = ax.twinx()
    ax_twin.set_ylim(0, 150)
    if i == 2:
        ax_twin.set_ylabel('Absolute Test Error/$\mathrm{\sigma}$ (%)')

In [ ]:
# We visualize the errors for different materials and selectivity values, in the case of ethylene selectivity target

# Load test data for ethylene selectivity prediction
test_data = get_test_data('./output/SISSO_results_publication/', 5, 2, 3, 'S_ethylene')

# Compute test errors
test_data["abserror"] = abs(test_data[1] - test_data[2])     # Absolute error
test_data["error"] = test_data[2] - test_data[1]             # Signed error

# Extract catalyst identity for color coding
test_data["labels"] = test_data[0].str[:4]                   # First 4 characters e.g., 'PdAg'

# Define color map and assign colors based on catalyst type
color_map = {'PdAg': 'grey', 'PdAu': 'darkorange', 'PdCu': 'brown'}
colors = test_data["labels"].map(color_map)

# Create side-by-side scatter plots ===
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), constrained_layout=True)
marker_size = 10

# Plot 1: SISSO prediction vs true value (Parity plot)
ax1.scatter(test_data[1], test_data[2], c=colors, s=marker_size)
ax1.plot([-3, 2], [-3, 2], color='k', linestyle='--', linewidth=1)  # Diagonal reference line
ax1.set_xlabel('$S_{\mathrm{C_2H_4}}$')                             # True value
ax1.set_ylabel('$S_{\mathrm{C_2H_4}}^{\mathrm{SISSO}}$')            # Predicted value
ax1.set_xlim(-2.6, 1)
ax1.set_ylim(-2.6, 1)

# Plot 2: Absolute error vs true value
ax2.scatter(test_data[1], test_data["abserror"], c=colors, s=marker_size)
ax2.set_xlabel('$S_{\mathrm{C_2H_4}}$')
ax2.set_ylabel('Absolute Test Error')

In [ ]:
# Visualization of model fit to the data 
# We look at the models trained on the entire dataset 
# at the optimal complexity identified by cross-validation
targets=performance_targets[1:]
for i in [0,1,2]:
    df_fit=pd.read_csv('./output/SISSO_results_publication/full_dataset/'+targets[i]+'_r'+str(optimal_rungs[i])+'/models/train_dim_'+str(optimal_dimensions[i])+'_model_0.dat',
                        comment='#',
                        names=['material_Temperature_time',
                            'target_true','target_fit',
                            'feature_0','feature_1','feature_2']).set_index('material_Temperature_time')
    df_SGD_selected[targets[i]+'_SISSO']=df_fit['target_fit']

In [ ]:
# Configure global plot font size
plt.rcParams.update({'font.size': 15})

# Create 3x2 grid of subplots (for three products at two temperatures)
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(12, 9), constrained_layout=True)

# Define visual properties for catalysts
colors = ['grey'] * 3 + ['darkorange'] * 3 + ['brown'] * 3
markers = ['s', '>', 'o'] * 3
marker_size = 30

# Loop over each of the 9 catalysts
for i in range(9):    
    # Get time-series data for current catalyst at 100°C and 150°C
    cat_100 = df_SGD_selected.loc[df_SGD_selected['material_Temperature'] == catalyst_list[i] + '_100']
    cat_150 = df_SGD_selected.loc[df_SGD_selected['material_Temperature'] == catalyst_list[i] + '_150']
    
    # Scatter plot for S_ethylene
    ax1.scatter(cat_100['time'], cat_100['S_ethylene'], color=colors[i], s=marker_size, marker=markers[i])
    ax1.scatter(cat_100['time'], cat_100['S_ethylene_SISSO'], color=colors[i], s=marker_size, marker='x')
    ax2.scatter(cat_150['time'], cat_150['S_ethylene'], color=colors[i], s=marker_size, marker=markers[i])
    ax2.scatter(cat_150['time'], cat_150['S_ethylene_SISSO'], color=colors[i], s=marker_size, marker='x')
    
    # Scatter plot for S_ethane
    ax3.scatter(cat_100['time'], cat_100['S_ethane'], color=colors[i], s=marker_size, marker=markers[i])
    ax3.scatter(cat_100['time'], cat_100['S_ethane_SISSO'], color=colors[i], s=marker_size, marker='x')
    ax4.scatter(cat_150['time'], cat_150['S_ethane'], color=colors[i], s=marker_size, marker=markers[i])
    ax4.scatter(cat_150['time'], cat_150['S_ethane_SISSO'], color=colors[i], s=marker_size, marker='x')
    
    # Scatter plot for S_C4
    ax5.scatter(cat_100['time'], cat_100['S_C4'], color=colors[i], s=marker_size, marker=markers[i])
    ax5.scatter(cat_100['time'], cat_100['S_C4_SISSO'], color=colors[i], s=marker_size, marker='x')
    ax6.scatter(cat_150['time'], cat_150['S_C4'], color=colors[i], s=marker_size, marker=markers[i])
    ax6.scatter(cat_150['time'], cat_150['S_C4_SISSO'], color=colors[i], s=marker_size, marker='x')

# Set axes limits
for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
    ax.set_xlim(0, 406)
ax1.set_ylim(-1.5, 1.5)
ax2.set_ylim(-1.5, 1.5)
ax3.set_ylim(0, 2.1)
ax4.set_ylim(0, 2.1)
ax5.set_ylim(0, 0.2)
ax6.set_ylim(0, 0.2)

# Set axis labels
ax1.set_ylabel('$S_{\mathrm{C_2H_4}}$')
ax3.set_ylabel('$S_{\mathrm{C_2H_6}}$')
ax5.set_ylabel('$S_{\mathrm{C4}}$')
ax5.set_xlabel('$t_{\mathrm{OS}}$ (min)')
ax6.set_xlabel('$t_{\mathrm{OS}}$ (min)')

# Tweak tick labels visibility for cleaner layout
for ax in [ax1, ax2, ax3, ax4]:
    ax.set_xticklabels([])
for ax in [ax2, ax4, ax6]:
    ax.set_yticklabels([])

# Add subplot titles for temperature conditions
ax1.set_title('$T_{\mathrm{oven}} = 100\,^\circ\mathrm{C}$')
ax2.set_title('$T_{\mathrm{oven}} = 150\,^\circ\mathrm{C}$')

# Add horizontal reference line at y=0 to the top row
ax1.axhline(0, color='k', linewidth=0.5)
ax2.axhline(0, color='k', linewidth=0.5)

# Add legend for catalysts (only once)
ax2.legend(
    labels=['measured', 'SISSO'],
    loc=(1.04, 0.05),
    frameon=True,
    ncol=1,
    fontsize=10)

In [ ]:
# Extract the model expressions
# We look at the models trained on the entire dataset 
# at the optimal complexity identified by cross-validation

for i in [0,1,2]:
    print('\n SISSO Model for ', targets[i])
    with open('./output/SISSO_results_publication/full_dataset/'+targets[i]+'_r'+str(optimal_rungs[i])+'/models/train_dim_'+str(optimal_dimensions[i])+'_model_0.dat', 'r') as file:
         for j, line in enumerate(file, start=1):
            if j in [1,4,5,6,7]:
                print(line, end='')
            if j >= 7:
                break

In [ ]:
# Extract the model expressions of the top-50 ranked models for ethylene selectivity
for i in range(50):
    print('\n Top '+str(i)+' SISSO Model for ', targets[0])
    with open('./output/SISSO_results_publication/full_dataset/'+targets[0]+'_r'+str(optimal_rungs[0])+'_50_output_models/models/train_dim_'+str(optimal_dimensions[0])+'_model_'+str(i)+'.dat', 'r') as file:
         for j, line in enumerate(file, start=1):
            if j in [1,3,4,5,6,7]:
                print(line, end='')
            if j >= 7:
                break

# 5. Exploiting the SGD and SISSO Models to Design New Materials

In [ ]:
# Load dataset of elemental properties 
df_elemental=pd.read_csv('./data/elemental_properties.csv').set_index('element')
df_elemental

In [ ]:
# Create candidate descriptive parameters (primary features) for hypothetical materials
# obtained with new elements mixed with palladium at different stoichiometries

elements=df_elemental.index.tolist()
elements.remove('Pd')

# Chosen range of stoichiometries
x_element=np.linspace(0.1, 0.9, 9).tolist()

elemental_features=['IP', 
                    'EA', 
                    'PE',
                    'bulk_int_dist', 
                    'E_coh', 
                    'B_0', 
                    'mu_C_critical_surf',
                    'mu_C_critical_subsurf', 
                    'E_b_C_subsurf', 
                    'E_def_subsurf',
                    'delta_M1_M2_subsurf', 
                    'BE_H_surf', 
                    'W_change_H',
                    'd_band_center_surf', 
                    'BE_H_subsurf']

rows = []

for element in elements:
    for x in x_element:
        x_str = f"{x:.1f}" 
        x_comp_str = f"{(1-x):.1f}"
        row={}
        row['material']='Pd'+x_comp_str+element+x_str
        for parameter in elemental_features:
            feature_value=df_elemental.at[element, parameter]*x+df_elemental.at['Pd', parameter]*(1-x)
            row['av_'+parameter]=feature_value
        rows.append(row)
           
df_new_bimetallic = pd.DataFrame(rows).set_index('material')

In [ ]:
# Visuzalize the hypothetical materials in the coordinates of the key materials-parameters identified by SGD
# The SG rules are also shown

fig, (ax1) = plt.subplots(1,1, constrained_layout=True, figsize=(4,4))
marker_size=30
ax1.vlines(4.6685,2.5659,3.05,color='red',linestyle='dashed')
ax1.hlines(2.5659,4.6685,9,color='red',linestyle='dashed')
ax1.fill_between([4.6685,9], 2.5659, 3.05, color='mistyrose')
ax1.fill([7.8,6.05,4.33], [2.753,2.556,2.889], color='orange',  edgecolor='orange')

ax1.scatter(df_SGD['av_E_b_C_subsurf'],df_SGD['av_bulk_int_dist'],c=color_entire_dataset,s=marker_size)
ax1.scatter(df_SGD_selected['av_E_b_C_subsurf'],df_SGD_selected['av_bulk_int_dist'],c=color_SG,s=marker_size)
ax1.scatter(df_new_bimetallic['av_E_b_C_subsurf'],df_new_bimetallic['av_bulk_int_dist'],c='grey',s=marker_size)

ax1.set_ylim(2.4,3.05)
ax1.set_xlim(4,8.5)
ax1.arrow(4.6685, 2.7, 0.5, 0, head_width=0.015, head_length=0.2, color=color_SG)
ax1.arrow(5.8, 2.5659, 0, 0.06, head_width=0.10, head_length=0.03, color=color_SG)

ax1.set_xlabel('$\\widebar{E_{\mathrm{b,C}}^{\mathrm{sub}}}$ (eV)')  
ax1.set_ylabel('$\\widebar{d_{\mathrm{closest}}}$ ($\mathrm{\AA}$)') 

In [ ]:
# Function to evaluate the SISSO model for ethylene selectivity
def evaluate_SISSO_model_ethylene_selectivity(av_W_change_H,
                                              av_d_band_center_surf,
                                              av_BE_H_surf, 
                                              av_E_def_subsurf, 
                                              total_metal_loading, 
                                              particle_diameter,
                                              av_BE_H_subsurf,
                                              time, 
                                              temperature):
    
    if temperature == 150:
        a=[4.813130462568657e-07, 3.727722063287428e+01,  -2.850231650770238e+00, -8.362104387720386e-02]
    if temperature == 100:
        a=[1.263789736634742e-06, 5.269016813835863e+01,  -2.661969356874947e+00, -1.704277305893994e-01]
    
    term_1=a[0]*(time / av_W_change_H) / (np.abs(av_d_band_center_surf - av_BE_H_surf))
    term_2=a[1]*(av_E_def_subsurf * total_metal_loading * particle_diameter)
    term_3=a[2]*(np.exp(8.365003e-04*((time+3.592461e+02) / (av_BE_H_subsurf+1.384560e+00))))
    selectivity=term_1+term_2+term_3+a[3]
    return(selectivity)

In [ ]:
# Visualize the predicted ethylene selectivity for new hypothetical bimetallic materials

# The experimental parameters entering the SISSO model are unknown for new materials
# We assume that the metal loading and mean particle diameter of these materials are
# equal to the mean values among the materials in the training set
mean_total_metal_loading=0.011755658627087199
mean_particle_diameter=5.667903525046382

# One temperature and two time-on-stream values are chosen
temperature = 150  
time = [40,400] 

# Chosen range of stoichiometries
x_element=[0.45, 0.55, 0.65, 0.75, 0.85, 0.90]

rows = []
selectivities_40_min = []
selectivities_400_min = []

for element in elements:
    selectivity_40_min=[]
    selectivity_400_min=[]
    for x in x_element:
        x_str = f"{x:.1f}" 
        x_comp_str = f"{(1-x):.1f}"
        row={}
        row['material']='Pd'+x_comp_str+element+x_str
        for parameter in elemental_features:
            feature_value=df_elemental.at[element, parameter]*x+df_elemental.at['Pd', parameter]*(1-x)
            row['av_'+parameter]=feature_value
        row['S_ethylene_SISSO_40min_150C']=evaluate_SISSO_model_ethylene_selectivity(row['av_W_change_H'],
                                                                                     row['av_d_band_center_surf'],
                                                                                     row['av_BE_H_surf'],
                                                                                     row['av_E_def_subsurf'],
                                                                                     mean_total_metal_loading,
                                                                                     mean_particle_diameter,
                                                                                     row['av_BE_H_subsurf'],
                                                                                     time[0],
                                                                                     temperature)
        row['S_ethylene_SISSO_400min_150C']=evaluate_SISSO_model_ethylene_selectivity(row['av_W_change_H'],
                                                                                      row['av_d_band_center_surf'],
                                                                                      row['av_BE_H_surf'],
                                                                                      row['av_E_def_subsurf'],
                                                                                      mean_total_metal_loading,
                                                                                      mean_particle_diameter,
                                                                                      row['av_BE_H_subsurf'],
                                                                                      time[1],
                                                                                      temperature)
        rows.append(row)
        selectivity_40_min.append(row['S_ethylene_SISSO_40min_150C'])
        selectivity_400_min.append(row['S_ethylene_SISSO_400min_150C'])
    selectivities_40_min.append(selectivity_40_min)
    selectivities_400_min.append(selectivity_400_min)

df_new_bimetallic = pd.DataFrame(rows).set_index('material')

x_element_labels=[45, 55, 65, 75, 85, 90]
    
fig, (ax1,ax2) = plt.subplots(1,2, constrained_layout=True, figsize=(8,4))
im = ax1.imshow(selectivities_40_min ,cmap='viridis',vmin=-1.35, vmax=0.96, aspect='auto')
im2 = ax2.imshow(selectivities_400_min ,cmap='viridis',vmin=-1.35, vmax=0.96, aspect='auto')
ax1.set_ylabel('element')
ax1.set_xlabel('$x_{\mathrm{element}}$ (%)')
ax1.set_xticks(ticks=[0,1,2,3,4,5], labels=x_element_labels)
ax1.set_yticks(ticks=[0,1,2,3,4,5,6,7,8], labels=elements)
fig.colorbar(im2, ax=ax2, label='Predicted $S_{\mathrm{C_2H_4}}^{\mathrm{SISSO}}$', fraction=0.050, pad=0.05)
ax2.set_xlabel('$x_{\mathrm{element}}$ (%)')
ax2.set_xticks(ticks=[0,1,2,3,4,5], labels=x_element_labels)
ax2.set_yticks(ticks=[0,1,2,3,4,5,6,7,8], labels='')
ax1.set_title('$t_{\mathrm{OS}}$ = 40 min')
ax2.set_title('$t_{\mathrm{OS}}$ = 400 min')

# 6. Experimental Verification of SISSO Model Predictions 

In [ ]:
# Load data
data_new = pd.read_csv('./data/performance_data_new_materials.csv')

# Set plotting parameters
plt.rcParams.update({'font.size': 16})
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), constrained_layout=True)
size = 20
time = np.arange(1, 400, 15)

# Define composition and physical parameters
compositions = [
    {'label': 'Pd_Ag_1_12_150', 'comp': {'Pd': 1/13, 'Ag': 12/13}, 'd': 6.03, 'm': 0.0153, 'ax': ax1, 'title': 'Pd$_1$Ag$_{12}$'},
    {'label': 'Pd_Ag_1_15_150', 'comp': {'Pd': 1/16, 'Ag': 15/16}, 'd': 7.04, 'm': 0.0161, 'ax': ax2, 'title': 'Pd$_1$Ag$_{15}$'},
    {'label': 'Pd_Ag_Cu_1_5_1_150', 'comp': {'Pd': 1/7, 'Ag': 5/7, 'Cu': 1/7}, 'd': 5.80, 'm': 0.0145, 'ax': ax3, 'title': 'Pd$_1$Ag$_5$Cu$_1$'}
]

# Loop over each composition
for cond in compositions:
    comp = cond['comp']
    ax = cond['ax']
    
    # Compute weighted average properties
    av_W_change_H = sum(comp[e] * df_elemental.at[e, 'W_change_H'] for e in comp)
    av_d_band_center_surf = sum(comp[e] * df_elemental.at[e, 'd_band_center_surf'] for e in comp)
    av_BE_H_surf = sum(comp[e] * df_elemental.at[e, 'BE_H_surf'] for e in comp)
    av_E_def_subsurf = sum(comp[e] * df_elemental.at[e, 'E_def_subsurf'] for e in comp)
    av_BE_H_subsurf = sum(comp[e] * df_elemental.at[e, 'BE_H_subsurf'] for e in comp)

    # Predict using SISSO model
    predictions = []
    for t in time:
        y = evaluate_SISSO_model_ethylene_selectivity(
            av_W_change_H,
            av_d_band_center_surf,
            av_BE_H_surf,
            av_E_def_subsurf,
            cond['m'],
            cond['d'],
            av_BE_H_subsurf,
            t,
            150
        )
        predictions.append(y)
    
    # Plot SISSO predictions
    ax.scatter(time, predictions, color='red', s=size, marker='s')
    
    # Plot experimental data
    data = data_new[data_new['material_Temperature'] == cond['label']]
    ax.scatter(data['time (min)'], data['X_acetylene'], color='k', s=size, marker='x')
    ax.scatter(data['time (min)'], data['S_ethylene'], color='blue', s=size, marker='s')
    
    # Axis formatting
    ax.set_xlim(0, 400)
    ax.set_ylim(0, 1.1)
    ax.hlines(0, 0, 400, color='k', linewidth=0.5)
    ax.set_xlabel('$t_{\mathrm{OS}}$(min)')
    ax.set_title(cond['title'])

# Add common y-label and clean up redundant ticks
ax1.set_ylabel('$S$ or $X$, $T_{\mathrm{oven}}=150\degree$C')
ax2.set_yticklabels([])
ax3.set_yticklabels([])

plt.show()

# 7. Additional Resources

Here are references of other works utilizing the SGD and SISSO approach. Each of them is associated with a dedicated jupyter notebook.


<div style="padding: 1ex; margin-top: 1ex; margin-bottom: 1ex; border-style: dotted; border-width: 1pt; border-color: blue; border-radius: 3px;"> L. Foppa and L. M. Ghiringhelli: <span style="font-style: italic;">Identifying Outstanding Transition-Metal-Alloy Heterogeneous Catalysts for the Oxygen Reduction and Evolution Reactions via Subgroup Discovery </span> <a href="https://link.springer.com/article/10.1007/s11244-021-01502-4" target="_top">Topics in Catalysis 65, 196 (2022)</a> <a href="https://nomad-lab.eu/AItutorials/sgd_alloys_oxygen_reduction_evolution" target="_top">[Jupyter Notebook]</a>
</div>

<div style="padding: 1ex; margin-top: 1ex; margin-bottom: 1ex; border-style: dotted; border-width: 1pt; border-color: blue; border-radius: 3px;"> L. Foppa, C. Sutton, L. M. Ghiringhelli, S. De, P. Loser, S. A. Schunk, A. Schafer, M. Scheffler: <span style="font-style: italic;">Learning Design Rules for Selective Oxidation Catalysts from High-Throughput Experimentation and Artificial Intelligence </span> <a href="https://doi.org/10.1021/acscatal.1c04793" target="_top">ACS Catalysis 12, 2223 (2022) </a> </span> <a href="https://nomad-lab.eu/AItutorials/SGD_Propylene_Oxidation_HTE" target="_top">[Jupyter Notebook]</a>
</div>

<div style="padding: 1ex; margin-top: 1ex; margin-bottom: 1ex; border-style: dotted; border-width: 1pt; border-color: blue; border-radius: 3px;">
L. Foppa and M. Scheffler: <span style="font-style: italic;">Coherent Collections of Rules Describing Exceptional Materials Identified with a Multi-Objective Optimization of Subgroups</span> <a href="https://arxiv.org/abs/2403.18437#" target="_blank">arXiv, 2403.18437 (2024)</a>  </span> <a href="https://github.com/lfoppa/Multi-objective-optimization-of-subgroups" target="_top">[Jupyter Notebook]</a>
</div>


<div style="padding: 1ex; margin-top: 1ex; margin-bottom: 1ex; border-style: dotted; border-width: 1pt; border-color: blue; border-radius: 3px;">
L. Foppa, T. A. R. Purcell, S. V. Levchenko, M. Scheffler, L. M. Ghiringhelli: <span style="font-style: italic;">Hierarchical Symbolic Regression for Identifying Key Physical Parameters Correlated with Bulk Properties of Perovskites </span> <a href="https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.129.055301" target="_blank">Physical Review Letters 129, 055301 (2022) </a>  </span> <a href="https://nomad-lab.eu/aitoolkit/hierarchical_sisso" target="_top">[Jupyter Notebook]</a>
</div>
</div>